# 03b - Record File Tab Data

In [23]:
import sys
!{sys.executable} -m pip install --upgrade pip -q
!{sys.executable} -m pip install pandas -q
!{sys.executable}  -m pip install numpy -q
!{sys.executable}  -m pip install matplotlib -q
!{sys.executable}   -m pip install scipy -q
#test comment

In [24]:
import numpy as np
import pandas as pd
import json
import os
from mgz import header, body
from mgz.model import parse_match
import pickle


In [3]:
def print_txt(msg: str):
    print(msg, end = '\r')
    text_file = open("Outputs/recordfile_tabdata.txt", "w")
    text_file.write(msg)
    text_file.close()

In [4]:
#load matches

with open('data/scraped_matches/parsed_matches/arabia_matches.pkl', 'rb') as f:
        matches = pickle.load(f)

matches_df = pd.DataFrame.from_dict(matches, orient='index')

matches_df.shape

(138902, 20)

In [5]:
matches_df.head()

,match_id,fn,map,map_size,duration,dataset,difficulty,input_fn,gaia_fn,p1_name,p2_name,p1_elo,p2_elo,p1_civ,p2_civ,p1_xpos,p2_xpos,p1_ypos,p2_ypos,winner
50453403,50453403,?gameId=50453403&profileId=594394,Arabia,Tiny,3445,Definitive Edition,Hardest,data/scraped_matches/inputs/50453403.pkl,data/scraped_matches/gaia_data/50453403.pkl,Zirafo,Esteban,975,1233,Vikings,Mayans,92.0,24.0,37.0,78.0,0
64335748,64335748,?gameId=64335748&profileId=1480279,Arabia,Tiny,851,Definitive Edition,Hardest,data/scraped_matches/inputs/64335748.pkl,data/scraped_matches/gaia_data/64335748.pkl,voltapette,Flood51,1154,1006,Mayans,Magyars,25.0,98.0,80.0,68.0,1
116883036,116883036,?gameId=116883036&profileId=3366080,Arabia,Tiny,4737,Definitive Edition,Hardest,data/scraped_matches/inputs/116883036.pkl,data/scraped_matches/gaia_data/116883036.pkl,KCube,santa777,984,1116,Berbers,Slavs,83.0,30.0,28.0,85.0,1
125287851,125287851,?gameId=125287851&profileId=3079077,Arabia,Tiny,2899,Definitive Edition,Hardest,data/scraped_matches/inputs/125287851.pkl,data/scraped_matches/gaia_data/125287851.pkl,TiM,BarDon,1708,1637,Vikings,Mayans,101.0,16.0,58.0,56.0,0
107928939,107928939,?gameId=107928939&profileId=2509560,Arabia,Tiny,1824,Definitive Edition,Hardest,data/scraped_matches/inputs/107928939.pkl,data/scraped_matches/gaia_data/107928939.pkl,TyphoonBD,AgeofPanda_21,1174,1096,Chinese,Japanese,46.0,84.0,97.0,28.0,1


In [25]:
#load masterdata
with open('masterdata/masterdata_dict.pkl', 'rb') as f:
        masterdata_dict = pickle.load(f)

        
#load masterdata
with open('masterdata/elo_frames.pkl', 'rb') as f:
        elo_frames = pickle.load(f)

In [7]:
error_list = []

In [8]:
#open ideas

##market activity (sell/buy0)
## wall 
## attack move
## garrison/town bell

In [9]:
def create_snapshot(inputs_dict: dict, time: int, masterdata_dict):
    snapshot = {}
    players = ['p1','p2']
    
    #create fields
    for player in players:
        for age in masterdata_dict['Ages']:
            snapshot[player+' '+age + ' Time'] = 0
            
        for cat in masterdata_dict:
            if cat != 'Ages': 
                for field in masterdata_dict[cat]:
                    snapshot[player+'_'+field] = 0
    
    #loop over inputs
    for index in inputs_dict:
        if inputs_dict[index]['ts_seconds'] <= time:
            if inputs_dict[index]['type'] == 'Research':

                #uptimes
                for age in masterdata_dict['Ages']:
                    if inputs_dict[index]['param'] == age:
                        snapshot[inputs_dict[index]['player']+' '+age + ' Time'] = inputs_dict[index]['ts_seconds'] 

            #sum up units, builds and techs
            for cat in masterdata_dict:
                if inputs_dict[index]['type'] == cat:
                        snapshot[inputs_dict[index]['player']+'_'+inputs_dict[index]['param']] +=1

                    
    return snapshot

In [10]:
matches_df['avg_elo'] = (matches_df['p1_elo'] + matches_df['p2_elo'] )/2
matches_df.shape

(138902, 21)

In [26]:
elo_frames

{'low': {'lb': 0, 'ub': 1062, 'label': 'Low Elo'},
 'medium': {'lb': 1062, 'ub': 1501, 'label': 'Medium Elo'},
 'high': {'lb': 1501, 'ub': 3000, 'label': 'High Elo'}}

In [27]:
#settings
time_list = [t for t in range(0,5121,120)]
time_reduction = False
time_start = 0

In [ ]:
for time in time_list:
    
    if time < time_start:
        continue
    
    
    if time_reduction:
        time_matches_df = matches_df[matches_df['duration']>=time]
    else:
        time_matches_df = matches_df
        
    #create empty dicts for Tab Data full and with elo steps
    tab_data = {}


    game_cols = ['map', 'map_size', 'duration', 'p1_civ', 'p2_civ', 'winner']
    count = 0
    errors = 0

    for index, row in time_matches_df.iterrows(): 
        
        if True: 
            with open(row['input_fn'], 'rb') as f:
                inputs_dict = pickle.load(f)

            #create row identifier
            row_id = str(row['match_id'])+'_'+str(time)
            tab_data[row_id] = {}

            #game info
            tab_data[row_id]['match_id'] = row['match_id']
            tab_data[row_id]['avg_elo'] = row['avg_elo']

            #set time
            tab_data[row_id]['time'] = time

            for col in game_cols:
                tab_data[row_id][col] = row[col]

            #create snapshot and fill row
            snapshot = create_snapshot(inputs_dict,time,masterdata_dict)
            for key in snapshot:
                tab_data[row_id][key] = snapshot[key]
        

            #output info
            count += 1

        else:#except:
            error_list.append(row['match_id'])
            errors +=1

        if count % 1000 == 0:
            print_txt(f'TF: {time}- Created: {count}/{len(time_matches_df)}- Error on:{errors}')

    
    #clean errors
    for match_id in error_list:
        if match_id in tab_data:
            tab_data.remove(match_id)        
            
    
    #persitate
    tab_data_df = pd.DataFrame.from_dict(tab_data, orient='index')
    output = open(f'data/minn/elo_all/recordfile_tab_data_{time}.pkl' , 'wb')
    pickle.dump(tab_data_df, output)
    

        

In [30]:
elo_frames

{'low': {'lb': 0, 'ub': 1062, 'label': 'Low Elo'},
 'medium': {'lb': 1062, 'ub': 1501, 'label': 'Medium Elo'},
 'high': {'lb': 1501, 'ub': 3000, 'label': 'High Elo'}}

In [42]:
for tf in time_list: 
        with open(f"data/minn/elo_all/recordfile_tab_data_{tf}.pkl", 'rb') as f:
            full_df = pickle.load(f)

        for elo_key in elo_frames:
            elo_tab_data_df =  full_df[(full_df['avg_elo']>= elo_frames[elo_key]['lb']) & (full_df['avg_elo']< elo_frames[elo_key]['ub'])]
            output = open(f'data/minn/elo_{elo_key}/recordfile_tab_data_{tf}.pkl' , 'wb')
            pickle.dump(elo_tab_data_df, output)
            print_txt(f'{tf} - Elo: {elo_key} - len {len(elo_tab_data_df)}       ')

In [ ]:
#cols = tab_data_df.columns.tolist()
#cols[0]

In [ ]:
#persistate
#output = open('masterdata/tab_data_cols.pkl' , 'wb')
#pickle.dump(cols, output)
#output.close()

In [ ]:
tab_data_df.shape

In [ ]:
tab_data_df.head()

In [ ]:
tab_data_df.columns

In [ ]:
#onehot
##map 
##mapsize
##civ


In [ ]:


#x = torch.tensor(["1", 3, 2, 1, 0])
#F.one_hot(x, num_classes=8)

In [ ]:
#x

In [ ]:
#F